In [ ]:
# Add project's path to system path
import os
import sys
try: 
    proj_path
except NameError:
    os.chdir('../')
    proj_path = os.getcwd()
    if proj_path not in sys.path:
        sys.path.append(proj_path)

# Import dependencies
import numpy as np
import matplotlib.pyplot as plt
import random
import networkx as nx
from scipy import linalg

from codebase import utils, rewiring, rewiring_iterations, directed_modularity # custom-made functions

## Functions

In [ ]:
def convert_from_adj2networkX(A,weight_d='binary'):

    edges_ind = np.where(A>0)
    num_edges = len(edges_ind[0])

    G = nx.DiGraph()   # DiGraph
    G.add_nodes_from(np.arange(A.shape[0]))

    edges_list = list()
    if weight_d == 'binary':
        for ind in np.arange(num_edges):
            edge_pair = (edges_ind[1][ind],edges_ind[0][ind])
            edges_list.append(edge_pair)
        
        G.add_edges_from(edges_list)    
    else:
        for ind in np.arange(num_edges):
            edge_pair_w = (edges_ind[1][ind],edges_ind[0][ind],A[edges_ind[0][ind],edges_ind[1][ind]])
            edges_list.append(edge_pair_w)    
        
        G.add_weighted_edges_from(edges_list)
        
        
    return G

In [ ]:
def getDigraphPathMetricsForHubs(Ax,inOutThresh = 10):

    inDeg = np.sum(Ax,axis=1)
    outDeg = np.sum(Ax,axis=0)
    
    #thresh = inOutThresh
    #numHubs = len(np.where(inDeg>=thresh)[0]) + len(np.where(outDeg>=thresh)[0])
    #ttt = 'num of hubs are '+str(numHubs)
    #print(ttt)
    
    #numHubs2 = len(np.union1d(np.where(inDeg>=thresh)[0],np.where(outDeg>=thresh)[0]))
    #ttt = 'num of hubs2 are '+str(numHubs2)
    #print(ttt)
    
    #bare minimum nodes should have greater than 1 in and out-degrees
    nonZeroIn = np.where(inDeg>1)[0]
    nonZeroOut = np.where(outDeg>1)[0]
    nonZeroInOut = np.intersect1d(nonZeroIn,nonZeroOut)

    inOutDeg = inDeg + outDeg
    indNodesTemp = np.where(inOutDeg>=inOutThresh)[0]
    indNodes2Use = np.intersect1d(nonZeroInOut,indNodesTemp)
    lenNodes2Use = len(indNodes2Use)
    
    
    G = convert_from_adj2networkX(Ax)

    paths = dict(nx.all_pairs_dijkstra_path(G))
    len_paths = dict(nx.all_pairs_dijkstra_path_length(G))


    # it is opposite from the adjacency matrix, i.e. pathsMatrix[i,j] is path length from i to j
    pathsMatrix = np.zeros((lenNodes2Use,lenNodes2Use)) 
    for indSource,source in enumerate(indNodes2Use):
        for target in len_paths[source].keys():
            if target in indNodes2Use:
            
                indTarget = np.where(indNodes2Use == target)[0][0]
                pathsMatrix[indSource, indTarget] = len_paths[source][target]

    numPaths = np.sum(pathsMatrix>0)
    numNonPaths = lenNodes2Use*(lenNodes2Use-1) -numPaths 
    distPaths = pathsMatrix[np.where(pathsMatrix>0)]
    
    return indNodes2Use, numPaths,numNonPaths,distPaths,pathsMatrix

In [ ]:
def getDigraphPathMetricsInOutConvDivUnit(Ax,inOutThresh):


    inDeg = np.sum(Ax,axis=1)
    outDeg = np.sum(Ax,axis=0)

    nonZeroIn = np.where(inDeg>1)[0]
    nonZeroOut = np.where(outDeg>1)[0]
    nonZeroInOut = np.intersect1d(nonZeroIn,nonZeroOut)

    inOutDeg = inDeg + outDeg
    indNodesTemp = np.where(inOutDeg>=inOutThresh)[0]
    indNodesUnit = np.intersect1d(nonZeroInOut,indNodesTemp)
    lenNodesUnit = len(indNodesUnit)

    indAllNodes = np.arange(100)
    #the nodes outside the convergent-divergent unit
    indNodesOut = np.setdiff1d(indAllNodes, indNodesUnit, assume_unique=False)


    #measure path lengths 
    G = convert_from_adj2networkX(Ax)
    paths = dict(nx.all_pairs_dijkstra_path(G))
    len_paths = dict(nx.all_pairs_dijkstra_path_length(G))


    #set up the matrix that will indicate the path from the out nodes to the unit and from the unit to the nodes
    nodes2unit = np.zeros((len(indNodesOut),len(indNodesUnit)))
    unit2nodes = np.zeros((len(indNodesUnit),len(indNodesOut)))

    #set up the matrix that will indicate the path from the out nodes to the unit 
    #it is opposite from the adjacency matrix, i.e. nodes2unit[i,j] is path length from i to j
    nodes2unit = np.zeros((len(indNodesOut),len(indNodesUnit)))
    for indSource,source in enumerate(indNodesOut):
        for indTarget,target in enumerate(indNodesUnit):
            if target in len_paths[source].keys():
                #tt = 'indTarget is '+str(indTarget)+' and indSource is '+str(indSource)
                #print(tt)
            
                nodes2unit[indSource, indTarget] = len_paths[source][target]

            
    #set up the matrix that will indicate the path from the unit to the out nodes 
    unit2nodes = np.zeros((len(indNodesUnit),len(indNodesOut)))
    for indSource,source in enumerate(indNodesUnit):
        for indTarget,target in enumerate(indNodesOut):
            if target in len_paths[source].keys():
                #tt = 'indTarget is '+str(indTarget)+' and indSource is '+str(indSource)
                #print(tt)
            
                unit2nodes[indSource, indTarget] = len_paths[source][target]            


    numPathsIn = np.sum(nodes2unit>0)
    numPathsOut = np.sum(unit2nodes>0)

    numNonPathsIn = nodes2unit.shape[0]*nodes2unit.shape[1] -numPathsIn 
    numNonPathsOut = unit2nodes.shape[0]*unit2nodes.shape[1] -numPathsOut 

    distPathsIn = nodes2unit[np.where(nodes2unit>0)]
    distPathsOut = unit2nodes[np.where(unit2nodes>0)]
    
    #at least one of the nodes in the unit sending to node out
    Out = np.sum(unit2nodes,axis=0)
    
    #the node out sending to at least one of the nodes in the unit
    In = np.sum(nodes2unit,axis=1)
    
    return indNodesUnit,indNodesOut,numPathsIn,numPathsOut,numNonPathsIn,numNonPathsOut,distPathsIn,distPathsOut,In,Out



In [ ]:
directory_load = 'data/'

flag = '_cons_adv_50_'

file_path_load = directory_load +'A'+flag+'pRand.pckl'
A = utils.load_var(file_path_load)

pRandVec = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8, 1]
repetitions = 100

#Ax = A[4][0.2,1][4000]

In [ ]:
inOutThresh = 15

inOutDict = {}
for rep in np.arange(repetitions):
    for indP,p in enumerate(pRandVec):
        
        Ax = A[rep+1][p,1][4000]
        indNodesUnit,indNodesOut,numPathsIn,numPathsOut,numNonPathsIn,numNonPathsOut,distPathsIn,distPathsOut,In,Out = getDigraphPathMetricsInOutConvDivUnit(Ax,inOutThresh)

        inOutDict[rep+1,p,flag] = (indNodesUnit,indNodesOut,numPathsIn,numPathsOut,numNonPathsIn,numNonPathsOut,distPathsIn,distPathsOut,In,Out)


In [ ]:
inOutDict[1,0.2,flag]